# Informationen für dieses Notebook

Dieses .ipnynb dient dazu, um die vorbereiteten Datensätze zu laden und Funktionen zu definieren um Filmvorschläge zu machen, welche der Nutzer bewerten soll. Am Ende soll eine Liste mit bewerteten FIlmen ausgegeben werden, welche für Machine-Learning Algorithmen eingesetzt werden soll, um vorhersagen für die Bewertung von unbekannten Filmen zu machen, die zu dem Filmgeschmack des Nutzers passen.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast  # Um Strings wieder in Listen zu konvertieren
import random

In [2]:
%matplotlib inline
sns.set_theme(style="darkgrid", palette="bright", context="paper")

In [3]:
# Maximale Anzahl an Spalten und Breite festlegen
pd.set_option('display.max_columns', None)  # Zeigt alle Spalten an
pd.set_option('display.width', 1000)  # Breite des Outputs in Zeichen

### CSV importieren (encodet & gescaled)

In [4]:
# CSV-Datei laden
df_test_similarity_scaled = pd.read_csv('df_test_similarity_scaled.csv')
df_test_similarity = pd.read_csv('df_test_similarity.csv')


In [5]:
import pickle

# Cosine Similarity-Matrix laden
with open("similarity_matrix.pkl", "rb") as f:
    cosine_similarity_matrix = pickle.load(f)

print("✅ Cosine Similarity-Matrix erfolgreich geladen!")


✅ Cosine Similarity-Matrix erfolgreich geladen!


### Funktion zum finden von Filmen die man wahrscheinlich kennt

In [6]:
def get_similar_movies_multi(movie_titles, df, similarity_matrix, top_n=10):
    """
    Findet eine Liste ähnlicher Filme basierend auf mehreren angegebenen Filmen.

    :param movie_titles: Liste der bekannten Filme des Nutzers
    :param df: DataFrame mit Filminformationen
    :param similarity_matrix: Precomputed Cosine Similarity Matrix
    :param top_n: Anzahl der auszugebenden Filme
    :return: DataFrame mit den Top-N ähnlichen Filmen
    """

    # Überprüfen, ob alle angegebenen Filme existieren
    valid_indices = []
    for title in movie_titles:
        if title in df['title'].values:
            valid_indices.append(df.index[df['title'] == title][0])
        else:
            print(f"⚠️ Warnung: '{title}' nicht im Datensatz gefunden und wird ignoriert!")

    if not valid_indices:
        print("❌ Keine bekannten Filme gefunden, Abbruch!")
        return None

    # Ähnlichkeiten für alle angegebenen Filme holen & in ein NumPy-Array umwandeln
    similarities = similarity_matrix[valid_indices].toarray()  # Fix: in Array konvertieren

    # Durchschnittliche Ähnlichkeit über alle angegebenen Filme berechnen
    mean_similarities = np.mean(similarities, axis=0)

    # Indizes der ähnlichsten Filme (sortiert nach höchster Ähnlichkeit)
    similar_indices = np.argsort(mean_similarities)[::-1]  # Absteigend sortieren

    # Überspringe die vom Nutzer angegebenen Filme selbst
    similar_indices = [idx for idx in similar_indices if idx not in valid_indices]

    # Top-N ähnliche Filme auswählen
    top_similar_movies = df.iloc[similar_indices[:top_n]].copy()
    top_similar_movies["similarity_score"] = mean_similarities[similar_indices[:top_n]]

    return top_similar_movies[['title', 'year', 'rating', 'similarity_score']]

In [7]:
# Beispielaufruf: Der Nutzer gibt mehrere Filme an
user_movies = ["Pulp Fiction", "The Matrix", "Inception", "The Dark Knight", "The Departed", "Interstellar", "No Country for Old Men", "The Revenant", "Dune: Part Two", "Dune: Part One"]

similar_movies = get_similar_movies_multi(user_movies, df_test_similarity_scaled, cosine_similarity_matrix, top_n=10)

# Ergebnis anzeigen
print(similar_movies)

                                           title      year    rating  similarity_score
18982                                  Sing Sing  1.645947  1.395421          0.891453
12727                          Top Gun: Maverick  1.592936  1.840283          0.889139
7633                               The Brutalist  1.698958  1.751310          0.889118
18901                           American Fiction  1.645947  1.217477          0.888945
20400                                       Room  1.221858  1.751310          0.888648
7553                                       Anora  1.698958  1.573366          0.888415
18904                              The Holdovers  1.645947  1.573366          0.887345
21760                      Manchester by the Sea  1.274869  1.484393          0.887244
2812   Three Billboards Outside Ebbing, Missouri  1.327880  1.751310          0.886850
7668                                 Nickel Boys  1.698958  1.306449          0.886505


## Sammeln von Informationen (Bewertungen)

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()
scaler.fit(df_test_similarity[['rating']])  # Das Training auf der originalen Rating-Spalte


StandardScaler()

In [10]:
def scale_user_rating(rating):
    """
    Skaliert eine Nutzerbewertung basierend auf dem bereits trainierten StandardScaler.
    """
    scaled_rating = scaler.transform(pd.DataFrame([[rating]], columns=['rating'])).flatten()[0]
    return scaled_rating


In [11]:
def rate_movie(movie_index, user_ratings, unknown_movies, rating=None):
    """
    Speichert die Bewertung eines Films oder markiert ihn als 'Kenne ich nicht'.
    
    :param movie_index: Index des Films im DataFrame
    :param user_ratings: Dictionary mit bewerteten Filmen {film_index: rating}
    :param unknown_movies: Set mit unbekannten Filmen
    :param rating: Bewertung (1-10) oder None falls "Kenne ich nicht" (Default: None)
    """
    if rating is None:
        unknown_movies.add(movie_index)  # Film wird als "kenne ich nicht" markiert
    else:
        scaled_rating = scale_user_rating(rating)  # **Nutzerbewertung skalieren**
        user_ratings[movie_index] = scaled_rating  # Speichere die Bewertung



In [12]:
def get_next_movie_suggestion(candidate_movie_indices, similarity_matrix, user_ratings, unknown_movies, cache_similarities=None):
    """
    Wählt den nächsten Film für die Bewertung aus, basierend auf aktuellen Bewertungen.
    Nutzt zwischengespeicherte Ähnlichkeitswerte für bessere Performance.
    
    :param candidate_movie_indices: Liste der möglichen Filme
    :param similarity_matrix: Cosine Similarity-Matrix
    :param user_ratings: Dictionary mit bereits bewerteten Filmen {film_index: rating}
    :param unknown_movies: Set mit Filmen, die der Nutzer nicht kennt
    :param cache_similarities: Optionaler Cache für Ähnlichkeitswerte
    """
    if not user_ratings:
        remaining_movies = [idx for idx in candidate_movie_indices if idx not in unknown_movies]
        return random.choice(remaining_movies) if remaining_movies else None

    rated_indices = np.array(list(user_ratings.keys()))
    rated_scores = np.array(list(user_ratings.values()))

    # Falls noch kein Cache existiert, wird er erstellt
    if cache_similarities is None:
        cache_similarities = similarity_matrix[:, rated_indices].toarray()

    # Gewichtete Summe der Bewertungen mit Ähnlichkeiten
    weighted_ratings = np.dot(cache_similarities, rated_scores) / (np.sum(cache_similarities, axis=1) + 1e-6)

    # Sortiere Filme nach höchster erwarteter Bewertung
    sorted_indices = np.argsort(weighted_ratings)[::-1]

    # Überspringe bereits bewertete & "Kenne ich nicht"-Filme
    for idx in sorted_indices:
        if idx in candidate_movie_indices and idx not in user_ratings and idx not in unknown_movies:
            return idx

    return None  # Falls keine Vorschläge mehr übrig sind




In [13]:
def interactive_movie_rating(df, similarity_matrix, user_movies, top_n=20):
    """
    Führt eine interaktive Filmbewertung durch und aktualisiert Vorschläge dynamisch.
    :param df: DataFrame mit Filminformationen
    :param similarity_matrix: Cosine-Similarity-Matrix
    :param user_movies: Liste der bekannten Filme des Nutzers
    :param top_n: Maximale Anzahl an Vorschlägen
    :return: DataFrame mit den bewerteten Filmen
    """
    global user_ratings, unknown_movies
    user_ratings = {}  # Speichert Nutzerbewertungen (Index -> Skalierte Bewertung)
    unknown_movies = set()  # Speichert Filme, die der Nutzer nicht kennt
    
    # Erste Vorauswahl basierend auf get_similar_movies_multi()
    similar_movies = get_similar_movies_multi(user_movies, df, similarity_matrix, top_n=top_n)
    if similar_movies is None or similar_movies.empty:
        print("❌ Keine passenden Filme gefunden! Wähle andere bekannte Filme.")
        return None
    
    # Indizes der Filme aus der ersten Vorauswahl
    candidate_movie_indices = similar_movies.index.tolist()
    
    print("\n🎬 Starte mit der Filmbewertung!")
    print("🔹 Gib 'exit' ein, um die Bewertung jederzeit abzubrechen.")

    # Dynamische Bewertungsrunde
    for i in range(top_n):
        print(f"\n🎬 Vorschlag {i + 1}/{top_n}")

        # Nächsten Film auswählen
        movie_idx = get_next_movie_suggestion(candidate_movie_indices, similarity_matrix, user_ratings, unknown_movies)
        if movie_idx is None:
            print("❌ Keine weiteren Vorschläge verfügbar!")
            break
        
        movie = df.iloc[movie_idx]
        
        print(f"📽️ Film: {movie['title']} ({movie['year']})")
        print(f"⭐ Bewertung im Datensatz: {movie['rating']:.2f}")

        # Nutzerbewertung abfragen
        while True:
            user_input = input("Wie bewertest du diesen Film? (1-10, oder 0 = Kenne ich nicht) ")
            
            if user_input.lower() == "exit":
                print("\n🚪 Bewertung abgebrochen. Deine bisherigen Bewertungen wurden gespeichert.")
                return pd.DataFrame.from_dict(user_ratings, orient='index', columns=['user_rating_scaled']).reset_index().merge(df[['title', 'year']], left_on='index', right_index=True)

            try:
                user_rating = int(user_input)
                
                if user_rating == 0:
                    rate_movie(movie_idx, user_ratings, unknown_movies)  # Fix: Keine unnötige Parameterübergabe
                    break  # Gehe zum nächsten Vorschlag
                
                elif 1 <= user_rating <= 10:
                    rate_movie(movie_idx, user_ratings, unknown_movies, user_rating)  # Fix: Richtige Reihenfolge
                    break  # Gehe zum nächsten Vorschlag
                
                else:
                    print("⚠️ Ungültige Eingabe. Bitte eine Zahl zwischen 1 und 10 eingeben.")
            
            except ValueError:
                print("⚠️ Ungültige Eingabe. Bitte eine gültige Zahl eingeben.")

        # Nach der Bewertung Vorschläge aktualisieren
        print("\n⏳ Vorschläge werden aktualisiert...")

    return pd.DataFrame.from_dict(user_ratings, orient='index', columns=['user_rating_scaled']).reset_index().merge(df[['title', 'year']], left_on='index', right_index=True)



In [16]:
# Beispiel: Liste mit bekannten Filmen angeben
user_movies = ["Avengers: Endgame", "Dune: Part Two", "Inception", "The Dark Knight", "Avengers: Infinity War", "Gladiator"]

# Interaktive Bewertung starten
rated_movies = interactive_movie_rating(df_test_similarity_scaled, cosine_similarity_matrix, user_movies, top_n=20)

# Ergebnisse anzeigen (falls vorhanden)
if rated_movies is not None:
    print("\n📊 Deine bewerteten Filme:")

    print(rated_movies[['title', 'year', 'user_rating_scaled']])


🎬 Starte mit der Filmbewertung!
🔹 Gib 'exit' ein, um die Bewertung jederzeit abzubrechen.

🎬 Vorschlag 1/20
📽️ Film: Gone Girl (1.1688464229941435)
⭐ Bewertung im Datensatz: 1.75



⏳ Vorschläge werden aktualisiert...

🎬 Vorschlag 2/20
📽️ Film: Sing Sing (1.6459471694766603)
⭐ Bewertung im Datensatz: 1.40

⏳ Vorschläge werden aktualisiert...

🎬 Vorschlag 3/20
📽️ Film: Oppenheimer (1.6459471694766603)
⭐ Bewertung im Datensatz: 1.93

⏳ Vorschläge werden aktualisiert...

🎬 Vorschlag 4/20
📽️ Film: Black Panther (1.3808911992085957)
⭐ Bewertung im Datensatz: 1.04

⏳ Vorschläge werden aktualisiert...

🎬 Vorschlag 5/20
📽️ Film: The Brutalist (1.6989583635302732)
⭐ Bewertung im Datensatz: 1.75

⏳ Vorschläge werden aktualisiert...

🎬 Vorschlag 6/20
📽️ Film: RRR (1.5929359754230472)
⭐ Bewertung im Datensatz: 1.48

⏳ Vorschläge werden aktualisiert...

🎬 Vorschlag 7/20
📽️ Film: Interstellar (1.1688464229941435)
⭐ Bewertung im Datensatz: 2.29

⏳ Vorschläge werden aktualisiert...

🎬 Vorschlag 8/20
📽️ Film: Tenet (1.4869135873158212)
⭐ Bewertung im Datensatz: 1.04

⏳ Vorschläge werden aktualisiert...

🎬 Vorschlag 9/20
📽️ Film: Blade Runner 2049 (1.3278800051549824)
⭐ Bewertung 

In [17]:
print(rated_movies)

    index  user_rating_scaled                                title      year
0   15610            1.662338                            Gone Girl  1.168846
1   18838            2.552061                          Oppenheimer  1.645947
2   28015            0.772615                        Black Panther  1.380891
3   15605            3.441784                         Interstellar  1.168846
4    1874            0.772615                                Tenet  1.486914
5    2794            1.662338                    Blade Runner 2049  1.327880
6   20307            1.662338                         The Revenant  1.221858
7   19325            3.441784                               Avatar  0.903790
8   12728            3.441784             Avatar: The Way of Water  1.592936
9   18836            0.772615  Spider-Man: Across the Spider-Verse  1.645947
10  20774            1.662338                       Dune: Part One  1.539925
11  12727            1.662338                    Top Gun: Maverick  1.592936